# Movie Recommendation System

© Explore Data Science Academy

---
### Honour Code

I {**2307FTDS_team_NM3 Members**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Twitter Sentiment Classification

In today’s technology driven world, recommender systems are socially and economically critical for ensuring that individuals can make appropriate choices surrounding the content they engage with on a daily basis. One application where this is especially true surrounds movie content recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

With this context, EDSA is challenging you to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity.

2307FTDS_team_NM3 Members:
 1.
 2.
 3.
 4.
 5. Christelle Coetzee
 6.

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Introduction</a>

<a href=#two>2. Importing Packages</a>

<a href=#three>3. Loading Data</a>

<a href=#four>4. Exploratory Data Analysis (EDA)</a>

<a href=#five>5. Feature Engineering</a>

<a href=#six>6. Modeling</a>

<a href=#seven>7. Model Performance</a>

<a href=#eight>8. Model Explanations</a>

<a href=#nine>9. Conclusion</a>

 <a id="one"></a>
## 1. Introduction
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Introduction ⚡ |
| :--------------------------- |
| In this section, we will introduce our institution and provide our problem statement. |

<a id="two"></a>
## 2. Import Packages and Libaries
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: package importation⚡ |
| :--------------------------- |
| In this section we will import all the packages we will need in order to run our notebook. |

---

In [2]:
# Installations:

# Uncomment and run the following commands to install necessary libraries:

# Install Comet ML for experiment tracking
#!pip install comet_ml

# Install WordCloud for generating word clouds
#!pip install wordcloud

# Install Contractions for handling contractions in text data
#!pip install contractions

# Install CatBoost for CatBoost Classifier
#!pip install catboost

# Install XGBoost for XGBoost Classifier
#!pip install xgboost

# Install MLxtend for additional machine learning utilities
#!pip install mlxtend

# Install or upgrade TensorFlow for neural network models
#!pip install --upgrade tensorflow

In [3]:
# Import necessary libraries
import numpy as np  # Numerical operations library
import pandas as pd  # Data manipulation and analysis library
import matplotlib.pyplot as plt  # Plotting library
import seaborn as sns  # Data visualization library based on Matplotlib
%matplotlib inline
from wordcloud import WordCloud  # Word cloud generation library
import sweetviz as sv

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import Comet ML for experiment tracking
from comet_ml import Experiment  # Experiment tracking library

# Import Natural Language Toolkit (nltk) for text processing
import nltk  # Natural Language Toolkit
from nltk.corpus import stopwords  # Stopwords for text processing
from nltk.stem import WordNetLemmatizer  # Lemmatization for text processing
from nltk.tokenize import word_tokenize, TreebankWordTokenizer  # Tokenization for text processing
nltk.download('punkt')
nltk.download('wordnet')
import unicodedata  # Unicode character normalization
import contractions  # Expansion of contractions in text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # Text vectorization libraries

# Import regular expressions and string manipulation module
import re  # Regular expressions
import string  # String manipulation
import itertools  # Iteration tools
from collections import Counter  # Collection data type

# Libraries for data preparation and model building
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix  # Classification metrics
from sklearn.model_selection import train_test_split, GridSearchCV  # Train-test split and grid search
from sklearn.linear_model import LogisticRegression  # Logistic Regression classifier for machine learning
from sklearn.tree import DecisionTreeClassifier  # Decision Tree classifier for machine learning
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier for machine learning
from sklearn.svm import LinearSVC, SVC  # Support Vector Machine classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB  # Naive Bayes classifiers
from sklearn.ensemble import BaggingClassifier  # Bagging classifier
from sklearn.ensemble import ExtraTreesClassifier  # Extra Trees classifier
from sklearn.ensemble import VotingClassifier  # Voting classifier
from xgboost import XGBClassifier  # Efficient and flexible gradient boosting library
from catboost import CatBoostClassifier  # High-performance gradient boosting on decision trees library
from scipy.sparse import hstack  # Used for stacking sparse matrices horizontally
import pickle  # Serialization library
from sklearn.utils import resample  # Resampling tool
from sklearn import feature_selection  # Feature selection module
from sklearn.feature_selection import f_classif  # Feature selection using F-statistic
from mlxtend.feature_selection import SequentialFeatureSelector  # Sequential feature selection
from sklearn import preprocessing  # Data preprocessing
import pickle  # Serialization library
import tensorflow as tf  # TensorFlow for neural net
from tensorflow.keras.layers import Dense  # Dense layer in Keras
from tensorflow.keras.models import Sequential  # Sequential model in Keras
from tensorflow.keras.utils import to_categorical  # Conversion to categorical data

# Feature selection Libraries:
from sklearn.feature_selection import SelectKBest  # To reduce features
from sklearn.feature_selection import chi2  # Used to estimate which features are most impactful

# Global Constants for reproducibility and consistency
TRAIN_TEST_SPLIT_VAR = 0.2  # Ratio for train-test split
RAND_STATE = 42  # Random seed for reproducibility
MAX_TEXT_FEATURES = 5000  # Maximum number of text features for all algorithms with exceptions listed below
RAD_SVC_Text_Features = 1000  # Specifically for use in radial SVC
NN_Text_Features = 1000  # Specifically for use in neural net
XGB_Text_Features = 1000  # Specifically for use in XGBoost algorithm
EXTRA_Text_Features = 1000  # Specifically for use in Extra Trees algorithm
BAG_Text_Features = 1000  # Specifically for use in Bagging algorithm
VEC_MIN_WORD_TO_REMOVE = 3  # Remove words that occur less than this value in the dataset

# Flags for notebook Execution
COMET_FLAG = False  # To gauge whether to commit experiments to Comet ML
VECTORIZER_TO_USE = "tfidf"  # Chooses between TfIDF vectorizer or Count Vectorizer - accepted values are "tfidf" or "count"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\T460\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\T460\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<a id="three"></a>
## 3. Loading of Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: data loading ⚡ |
| :--------------------------- |
| In this section, we will load our datasets. |

---

Let's begin by loading our data into a DataFrame:

In [4]:
# Read training data from CSV file into a DataFrame
df_genome_scores = pd.read_csv('genome_scores.csv')

df_genome_tags = pd.read_csv('genome_tags.csv')

df_imdb_data = pd.read_csv('imdb_data.csv')

df_links = pd.read_csv('links.csv')

df_movies = pd.read_csv('movies.csv')

df_tags = pd.read_csv('tags.csv')

df_train = pd.read_csv('train.csv')

# Read test from CSV file into a DataFrame
df_test = pd.read_csv('test.csv')

<a id="four"></a>
## 4. Exploratory Data Analysis
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: EDA process ⚡ |
| :--------------------------- |
| In this section, we will perform an in-depth analysis of all the variables in the DataFrame. |

### **4.1. Data Overview:**

As we embark on the data overview phase, our objective is to delve into the foundational structure of the dataset. This exploration will encompass an examination of key aspects such as the dataset's shape, columns, and initial content, providing us with a comprehensive understanding of the basic characteristics inherent in our data.

#### **4.1.1. Data Investigation:**

To commence our exploration of the dataset, we initiate a thorough examination of a segment of our data, aiming to identify and analyze the key variables under consideration. This initial step is fundamental to understanding the landscape of our dataset and laying the groundwork for subsequent analyses.

Since we have multiple databases, let's take a closer look at each one and what they encompass.

##### *4.1.1.1. `df_genome_scores`:*

Let's commence by showcasing our dataset named `df_genome_scores`:

In [5]:
# Display the first few rows of the genome_score dataset
df_genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


This dataset contains information about both movieId and tagId, with the relevance score indicating the degree of relevance between the tag and the movie. The tag genome assesses the strength of each tag's applicability to each item, represented on a continuous scale from 0 to 1. In this scale, 0 signifies no applicability, while 1 indicates a very strong relevance of the tag to the item.

The relevance score is valuable for both content-based and collaborative models. It contributes to the effectiveness of content-based models by quantifying content similarity, and it enhances collaborative models by incorporating content-related features to improve recommendations. The dual impact of relevance scores makes them a valuable component in creating a comprehensive recommendation system that combines both content and collaborative filtering approaches.

In [6]:
# Display the shape (number of rows and columns) of the genome_score dataset
df_genome_scores.shape

(15584448, 3)

The dataset comprises 15,584,448 rows and encompasses 3 columns, indicating its substantial size. Given the vast scale of this dataset, it becomes imperative to explore techniques for dimensionality reduction to efficiently manage and analyze the data.

In [7]:
# Display information about the genome_scores dataset, including data types
df_genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15584448 entries, 0 to 15584447
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 356.7 MB


This database exclusively contains floating-point numbers or integers, making it compatible for constructing models.

In [30]:
# Display the count of missing values in each column of the genome_scores dataset
df_genome_scores.isnull().sum()

movieId      0
tagId        0
relevance    0
dtype: int64

There are no missing values in this dataset. 

In [8]:
analyze_report = sv.analyze(df_genome_scores)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


##### *4.1.1.2. `df_genome_tags`:*

Let's commence by showcasing our dataset named `df_genome_tags`:

In [31]:
df_genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


The dataset `df_genome_tags` furnishes user-assigned tag descriptions corresponding to the tag IDs found in the `df_genome_scores` dataset.

The dataset is more important for the content-based model. It enriches the content representation, enhances interpretability, and contributes to personalized recommendations by incorporating user-generated insights into the content features. This user-generated metadata is particularly valuable when aiming for a recommendation system that reflects both automated content analysis and user perceptions.

In [10]:
df_genome_tags.shape

(1128, 2)

The dataset comprises 1128 rows and encompasses 2 columns.

In [32]:
# Display information about the genome_tags dataset, including data types
df_genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


The dataset includes an integer column (`tagId`) and an object column (`tag`). As models cannot directly process language, the object column needs to undergo encoding before we proceed with training the model.

In [33]:
# Display the count of missing values in each column of the genome_tags dataset
df_genome_tags.isnull().sum()

tagId    0
tag      0
dtype: int64

There are no missing values in this dataset.

In [11]:
analyze_report = sv.analyze(df_genome_tags)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


##### *4.1.1.3. `df_imdb_data`:*

Let's commence by showcasing our dataset named `df_imdb_data`:

In [12]:
df_imdb_data.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


This dataset incorporates supplementary movie metadata extracted from IMDB, utilizing the information provided in the `df_links` dataset links. It encompasses details such as the cast, director, runtime, budget, and plot keywords associated with a specific movie, identified by the `movieId`.

 The dataset is more important for the content-based model. It significantly enhances the content representation, contributes to content similarity assessments, supports personalization, and ultimately improves the quality of recommendations based on the content characteristics of movies. This additional metadata is valuable in creating a content-based recommendation system that takes into account both automated content analysis and user-specific preferences.

In [13]:
df_imdb_data.shape

(27278, 6)

The dataset comprises 27,278 rows and encompasses 6 columns.

In [37]:
# Display information about the imdb_data dataset, including data types
df_imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        27278 non-null  int64  
 1   title_cast     17210 non-null  object 
 2   director       17404 non-null  object 
 3   runtime        15189 non-null  float64
 4   budget         7906 non-null   object 
 5   plot_keywords  16200 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.2+ MB


The dataset comprises four object columns that necessitate encoding. Additionally, there is observable presence of missing values within the dataset.

In [36]:
# Display the count of missing values in each column of the imdb_data dataset
df_imdb_data.isnull().sum()

movieId              0
title_cast       10068
director          9874
runtime          12089
budget           19372
plot_keywords    11078
dtype: int64

This dataset exhibits a considerable number of missing values. Specifically, there are 10,068 missing values in the `title_cast` column, 9,874 in the `director` column, 12,089 in the `runtime` column, 19,372 in the `budget` column, and 11,078 in the `plot_keywords` column.

In [14]:
analyze_report = sv.analyze(df_imdb_data)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


##### *4.1.1.4. `df_links`:*

Let's commence by showcasing our dataset named `df_links`:

In [15]:
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


This dataset provids a mapping between a Movie ID and associated IMDB and TMDB IDs.

The dataset is more important for the collaborative model. It enables cross-dataset linking, enhances user similarity assessments, and allows collaborative models to leverage information from multiple sources. While it may have some impact on content-based models by providing additional content features, its primary significance lies in facilitating collaborative filtering methods that benefit from a broader range of user preferences and movie details.

In [16]:
df_links.shape

(62423, 3)

The dataset comprises 62,423 rows and encompasses 3 columns.

In [38]:
# Display information about the links dataset, including data types
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  62423 non-null  int64  
 1   imdbId   62423 non-null  int64  
 2   tmdbId   62316 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB


This database exclusively contains floating-point numbers or integers, making it compatible for constructing models.

In [39]:
# Display the count of missing values in each column of the links dataset
df_links.isnull().sum()

movieId      0
imdbId       0
tmdbId     107
dtype: int64

The column `tmdbId` in this dataset exhibits 107 instances of missing values.

In [17]:
analyze_report = sv.analyze(df_links)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


##### *4.1.1.5. `df_movies`:*

Let's commence by showcasing our dataset named `df_movies`:

In [18]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


The dataset encompasses the movieId, the movie title (including the year of release), and the genres, which are presented as a pipe-separated list. The genres are chosen from the following categories:

Action
Adventure
Animation
Children's
Comedy
Crime
Documentary
Drama
Fantasy
Film-Noir
Horror
Musical
Mystery
Romance
Sci-Fi
Thriller
War
Western
(no genres listed)

The dataset is important for both content-based and collaborative models, but its impact is more pronounced in content-based recommendation systems. Genres play a central role in content representation, content similarity assessments, and providing personalized recommendations based on users' genre preferences. However, collaborative models also benefit from the genre information to enhance user similarity and diversify recommendations within user clusters. The dataset's contribution to content-based models is more direct and prominent due to its role in content representation and similarity assessments.

In [19]:
df_movies.shape

(62423, 3)

The dataset comprises 62,423 rows and encompasses 3 columns.

In [40]:
# Display information about the movies dataset, including data types
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [41]:
# Display the count of missing values in each column of the movies dataset
df_movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [20]:
analyze_report = sv.analyze(df_movies)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [21]:
df_tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [22]:
df_tags.shape

(1093360, 4)

In [23]:
analyze_report = sv.analyze(df_tags)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [24]:
df_train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [25]:
df_train.shape

(10000038, 4)

In [26]:
analyze_report = sv.analyze(df_train)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [27]:
df_test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [28]:
df_test.shape

(5000019, 2)

Collaborative-based dataset:

In [29]:
# Merge df_train and df_tags on 'movieId'
collaborative_df = pd.merge(df_train, df_tags, on='movieId', how='inner')

# Display the first few rows of the collaborative dataframe
collaborative_df.head()

MemoryError: Unable to allocate 32.4 GiB for an array with shape (4351252147,) and data type int64

<a id="five"></a>
## 5. Feature Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section we will clean the dataset, and create new features - as identified in the EDA phase. |

---

<a id="six"></a>
## 6. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, we will create models that are able to accurately recommend movies to users. |

---

<a id="seven"></a>
## 7. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section we will compare the relative performance of the various trained machine learning models on a holdout dataset and comment on what model is the best and why. |

---

<a id="eight"></a>
## 8. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, we will compare the models and determine the best model to use. We will also discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

<a id="nine"></a>
## 9. Conclusion
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, we will conclude the project. |

---